In [149]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,accuracy_score,confusion_matrix
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.manifold import MDS
from sklearn.preprocessing import Imputer,StandardScaler,MinMaxScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
import seaborn as sns
from __future__ import division
%matplotlib inline

In [150]:
X = pd.read_csv('S&P500_2017.csv')
X = X.drop(['Date'],axis=1)

In [151]:
X.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
0,46.074703,45.816051,170.237915,113.847588,59.784031,80.855621,37.982937,114.158501,103.480003,70.577911,...,36.900692,57.513260,86.670731,58.296070,26.833357,48.875042,61.848686,102.513084,42.586197,53.118965
1,46.679256,46.211868,171.634933,113.720161,60.626999,82.862099,38.284462,114.432961,104.139999,70.431908,...,37.027840,57.094593,85.717155,58.773418,27.845938,49.603493,62.073734,103.455551,43.197666,53.634392
2,46.124260,45.410336,171.515182,114.298462,61.086800,81.902901,38.615170,112.717552,105.910004,69.419624,...,36.607288,56.403305,84.439362,58.007671,27.729099,49.150673,62.279217,104.120262,42.497429,53.455975
3,47.561314,45.726990,169.269974,115.572708,61.105961,82.999107,39.665661,114.001671,108.300003,69.692154,...,36.568165,57.493786,84.391678,57.967899,27.300703,48.806133,63.032623,104.130180,42.773582,53.624477
4,47.709969,46.587898,169.170197,116.631294,61.508286,83.664680,39.626755,112.727356,108.570000,70.023094,...,35.952015,57.503525,82.999451,58.146900,27.144920,48.599411,63.208748,106.154053,42.310051,53.475800


In [162]:
r = 0.035/365
n = 10000

In [163]:
Times = [(t1,t1+np.random.choice(range(1,30))) for t1 in [np.random.choice(range(220))for _ in range(n)] ]

In [164]:
M=X.fillna(X.mean())
M = M.pct_change()
Z = pd.DataFrame(columns=M.columns)
Z = Z.T
Z['Mean'] = M.mean()
Z['Std'] = M.std()
Z['Skew'] = M.skew()
Z['Kurt'] = M.kurt()

$$K=S_0e^{rt}$$

In [165]:
Y = pd.DataFrame(columns=['Mean','Std','Skew','Kurt','I','F','D'],index=range(n))
for i in range(n):
    ran = np.random.choice(503)
    t = (Times[i][1]-Times[i][0])/365
    S0 = X.T.iloc[ran][Times[i][0]]
    K = S0*np.e**(r*t)
    St =  X.T.iloc[ran][Times[i][1]] 
    d=  (K <=St)*1
    Y.iloc[i]=list(Z.iloc[ran])+[Times[i][0],Times[i][1],d]

In [156]:
for col in Y.columns:
    Y[col] = Y[col].apply(pd.to_numeric, errors='coerce')

In [157]:
Y.describe()

,Mean,Std,Skew,Kurt,I,F,D
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000
mean,0.000720,0.013826,0.008961,9.795435,109.84900,124.467000,0.580000
std,0.001111,0.013690,1.596918,15.634585,61.26988,61.262762,0.493805
min,-0.002363,0.004954,-5.862934,-0.042685,0.00000,3.000000,0.000000
25%,0.000157,0.009866,-0.576259,2.129604,58.75000,72.000000,0.000000
50%,0.000723,0.012064,-0.071498,4.925315,112.00000,124.000000,1.000000
75%,0.001241,0.014798,0.555571,11.262785,159.00000,174.000000,1.000000
max,0.013025,0.260683,14.712657,229.653782,219.00000,245.000000,1.000000


In [158]:
W = Y[['Mean','Std','Skew','Kurt','I','F']]
y = Y[['D']]

In [159]:
Xt,Xv,yt,yv =train_test_split(W,y,train_size=0.7)
for d in [Xt,Xv,yt,yv]:
    print d.shape

(700, 6)
(300, 6)
(700, 1)
(300, 1)


In [160]:
def metricas(model,Xt,Xv,yt,yv):
    print roc_auc_score(y_score=model.predict_proba(Xt)[:,1],y_true=yt)
    print roc_auc_score(y_score=model.predict_proba(Xv)[:,1],y_true=yv)
    print accuracy_score(y_pred=model.predict(Xt),y_true=yt)
    print accuracy_score(y_pred=model.predict(Xv),y_true=yv)
    print confusion_matrix(y_pred=model.predict(Xt),y_true=yt)
    print confusion_matrix(y_pred=model.predict(Xv),y_true=yv)

In [161]:
model =LogisticRegression()
model.fit(Xt,yt)
metricas(model,Xt,Xv,yt,yv)

0.588696951383
0.538381177326
0.587142857143
0.56
[[ 49 243]
 [ 46 362]]
[[ 14 114]
 [ 18 154]]


/home/aleespa/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
